# 패키지, 파일 로딩

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
import geopy
import plotly.express as px
import plotly.graph_objects as go

In [2]:
from geoband.API import *
GetCompasData('SBJ_2208_001', '15', '15_연수구_아파트_매매_실거래가.xlsx')
GetCompasData('SBJ_2208_001', '16', '16_연수구_연립다세대_매매_실거래가.xlsx')
GetCompasData('SBJ_2208_001', '18', '18_연수구_오피스텔_매매_실거래가.xlsx')

GetCompasData('SBJ_2208_001', '19', '19_연수구_아파트_전월세_실거래가.xlsx')
GetCompasData('SBJ_2208_001', '20', '20_연수구_연립다세대_전월세_실거래가.xlsx')
GetCompasData('SBJ_2208_001', '22', '22_연수구_오피스텔_전월세_실거래가.xlsx')

GetCompasData('SBJ_2208_001', '24', '24_연수구_토지_실거래가.xlsx')
GetCompasData('SBJ_2208_001', '25', '25_연수구_분양입주권_실거래가.xlsx')

[2022-09-30 16:45:41] 성공 - 15_연수구_아파트_매매_실거래가.xlsx
[2022-09-30 16:45:41] 성공 - 16_연수구_연립다세대_매매_실거래가.xlsx
[2022-09-30 16:45:42] 성공 - 18_연수구_오피스텔_매매_실거래가.xlsx
[2022-09-30 16:45:42] 성공 - 19_연수구_아파트_전월세_실거래가.xlsx
[2022-09-30 16:45:42] 성공 - 20_연수구_연립다세대_전월세_실거래가.xlsx
[2022-09-30 16:45:42] 성공 - 22_연수구_오피스텔_전월세_실거래가.xlsx
[2022-09-30 16:45:42] 성공 - 24_연수구_토지_실거래가.xlsx
[2022-09-30 16:45:42] 성공 - 25_연수구_분양입주권_실거래가.xlsx


'25_연수구_분양입주권_실거래가.xlsx'

In [3]:
apart_trade = pd.read_excel('15_연수구_아파트_매매_실거래가.xlsx')
villa_trade = pd.read_excel('16_연수구_연립다세대_매매_실거래가.xlsx')
officetel_trade = pd.read_excel('18_연수구_오피스텔_매매_실거래가.xlsx')

In [4]:
apart_rent = pd.read_excel('19_연수구_아파트_전월세_실거래가.xlsx')
villa_rent = pd.read_excel('20_연수구_연립다세대_전월세_실거래가.xlsx')
officetel_rent = pd.read_excel('22_연수구_오피스텔_전월세_실거래가.xlsx')

In [5]:
land_trade = pd.read_excel('24_연수구_토지_실거래가.xlsx')
residency_trade = pd.read_excel('25_연수구_분양입주권_실거래가.xlsx')

# 전처리 - 15_연수구_아파트_매매_실거래가

In [6]:
apart_trade.head()

,bjd_nm,road_nm,bunji,cmplx_nm,exclusive_area,cntrc_ym,cntrc_day,trade_price,floor,cnstr_year
0,인천광역시 연수구 동춘동,먼우금로83번길 12,938,건영,84.945,202201,25,47000,1,1994.0
1,인천광역시 연수구 동춘동,먼우금로83번길 12,938,건영,84.945,202202,9,38000,1,1994.0
2,인천광역시 연수구 동춘동,먼우금로83번길 12,938,건영,84.945,202204,30,42200,5,1994.0
3,인천광역시 연수구 동춘동,먼우금로83번길 12,938,건영,73.660,202205,22,39200,5,1994.0
4,인천광역시 연수구 동춘동,먼우금로83번길 12,938,건영,84.945,202206,3,45500,2,1994.0


In [7]:
apart_trade.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56711 entries, 0 to 56710
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   bjd_nm          56711 non-null  object 
 1   road_nm         56711 non-null  object 
 2   bunji           56711 non-null  object 
 3   cmplx_nm        56711 non-null  object 
 4   exclusive_area  56711 non-null  float64
 5   cntrc_ym        56711 non-null  int64  
 6   cntrc_day       56711 non-null  int64  
 7   trade_price     56711 non-null  int64  
 8   floor           56711 non-null  int64  
 9   cnstr_year      56709 non-null  float64
dtypes: float64(2), int64(4), object(4)
memory usage: 4.3+ MB


In [8]:
apart_trade[apart_trade['cnstr_year'].isnull()]
# 해당 아파트는 2021년 입주 (출처: https://hogangnono.com/apt/dHr2a) 
# -> cnstr_year = 2021 채워넣기

,bjd_nm,road_nm,bunji,cmplx_nm,exclusive_area,cntrc_ym,cntrc_day,trade_price,floor,cnstr_year
205,인천광역시 연수구 송도동,인천타워대로231번길 117,114,더샵송도프라임뷰25BL,121.18,202203,31,90000,3,NaN
206,인천광역시 연수구 송도동,인천타워대로231번길 117,114,더샵송도프라임뷰25BL,121.18,202204,27,94000,3,NaN


In [9]:
apart_trade = apart_trade.fillna(2021)

In [10]:
dong = []
for i in apart_trade['bjd_nm']:
    dong.append(i[-3:])
apart_trade['dong'] = dong

cntrc = []
for i in apart_trade['cntrc_ym']:
    cntrc.append(str(i)[0:4])

cntrc_year = []
for year in cntrc:
    cntrc_year.append(int(year))
apart_trade['cntrc_year'] = cntrc_year

price_per_area = [] # 제곱미터당 가격
for i in range(len(apart_trade)):
    price_per_area.append( apart_trade['trade_price'][i] / apart_trade['exclusive_area'][i] )
apart_trade['price_per_area'] = price_per_area

In [11]:
apart_trade.head()

,bjd_nm,road_nm,bunji,cmplx_nm,exclusive_area,cntrc_ym,cntrc_day,trade_price,floor,cnstr_year,dong,cntrc_year,price_per_area
0,인천광역시 연수구 동춘동,먼우금로83번길 12,938,건영,84.945,202201,25,47000,1,1994.0,동춘동,2022,553.299194
1,인천광역시 연수구 동춘동,먼우금로83번길 12,938,건영,84.945,202202,9,38000,1,1994.0,동춘동,2022,447.348284
2,인천광역시 연수구 동춘동,먼우금로83번길 12,938,건영,84.945,202204,30,42200,5,1994.0,동춘동,2022,496.792042
3,인천광역시 연수구 동춘동,먼우금로83번길 12,938,건영,73.660,202205,22,39200,5,1994.0,동춘동,2022,532.174857
4,인천광역시 연수구 동춘동,먼우금로83번길 12,938,건영,84.945,202206,3,45500,2,1994.0,동춘동,2022,535.640709


In [12]:
apart_trade = apart_trade[(apart_trade['cntrc_year'] == 2018)
                          | (apart_trade['cntrc_year'] == 2019)
                          | (apart_trade['cntrc_year']==2020)
                          | (apart_trade['cntrc_year']==2021)]
apart_trade.reset_index(inplace=True)
apart_trade.drop('index', axis=1, inplace=True)
apart_trade.head()

,bjd_nm,road_nm,bunji,cmplx_nm,exclusive_area,cntrc_ym,cntrc_day,trade_price,floor,cnstr_year,dong,cntrc_year,price_per_area
0,인천광역시 연수구 동춘동,먼우금로83번길 12,938,건영,84.945,201801,29,24000,5,1994.0,동춘동,2018,282.535758
1,인천광역시 연수구 동춘동,먼우금로83번길 12,938,건영,73.660,201802,13,23800,2,1994.0,동춘동,2018,323.106163
2,인천광역시 연수구 동춘동,먼우금로83번길 12,938,건영,84.945,201802,19,25300,3,1994.0,동춘동,2018,297.839779
3,인천광역시 연수구 동춘동,먼우금로83번길 12,938,건영,84.945,201802,24,24000,5,1994.0,동춘동,2018,282.535758
4,인천광역시 연수구 동춘동,먼우금로83번길 12,938,건영,84.945,201802,24,24800,1,1994.0,동춘동,2018,291.953617


## 최종 데이터프레임
* base_date: 계약연도 (cntrc_year)

In [13]:
apart_trade_final = pd.DataFrame({'end_nm': apart_trade['dong'], 'base_date': apart_trade['cntrc_year'],
                                  'exclusive_area': apart_trade['exclusive_area'], 'price_per_area': apart_trade['price_per_area'].round(3),
                                   'cnstr_year': apart_trade['cnstr_year'].astype(int), 'floor': apart_trade['floor']
                                   })

In [14]:
apart_trade_final['bld_type'] = '아파트'

In [15]:
apart_trade_final.head()

,end_nm,base_date,exclusive_area,price_per_area,cnstr_year,floor,bld_type
0,동춘동,2018,84.945,282.536,1994,5,아파트
1,동춘동,2018,73.660,323.106,1994,2,아파트
2,동춘동,2018,84.945,297.840,1994,3,아파트
3,동춘동,2018,84.945,282.536,1994,5,아파트
4,동춘동,2018,84.945,291.954,1994,1,아파트


# 전처리 - 16_연수구_연립다세대_매매_실거래가

In [16]:
villa_trade.head()

,bjd_nm,road_nm,bunji,bld_nm,exclusive_area,land_area,cntrc_ym,cntrc_day,trade_price,floor,cnstr_year
0,인천광역시 연수구 선학동,넘말로47번길 37,393-7,"구룡마을(103,104)동",54.08,26.75,201301,2,9000,4,1995
1,인천광역시 연수구 옥련동,청룡로 40,348-160,한샘빌라(348-160),36.42,21.97,201301,17,5000,1,1991
2,인천광역시 연수구 청학동,청학로12번길 29,535-4,금잔디빌라(535-4),35.70,27.20,201301,18,4000,1,1990
3,인천광역시 연수구 청학동,청학로6번길 85,550-15,백학맨션8차(550-15),53.80,29.94,201301,8,8300,1,1996
4,인천광역시 연수구 청학동,청학안로 12,519-5,산장타운(519-5),35.10,22.45,201301,4,7500,3,1995


In [17]:
villa_trade.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5407 entries, 0 to 5406
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   bjd_nm          5407 non-null   object 
 1   road_nm         5407 non-null   object 
 2   bunji           5407 non-null   object 
 3   bld_nm          5407 non-null   object 
 4   exclusive_area  5407 non-null   float64
 5   land_area       5407 non-null   float64
 6   cntrc_ym        5407 non-null   int64  
 7   cntrc_day       5407 non-null   int64  
 8   trade_price     5407 non-null   int64  
 9   floor           5407 non-null   int64  
 10  cnstr_year      5407 non-null   int64  
dtypes: float64(2), int64(5), object(4)
memory usage: 464.8+ KB


In [18]:
dong = []
for i in villa_trade['bjd_nm']:
    dong.append(i[-3:])
villa_trade['dong'] = dong

cntrc = []
for i in villa_trade['cntrc_ym']:
    cntrc.append(str(i)[0:4])

cntrc_year = []
for year in cntrc:
    cntrc_year.append(int(year))
villa_trade['cntrc_year'] = cntrc_year

price_per_area = [] # 제곱미터당 가격
for i in range(len(villa_trade)):
    price_per_area.append( villa_trade['trade_price'][i] / villa_trade['exclusive_area'][i] )
villa_trade['price_per_area'] = price_per_area

In [19]:
villa_trade.head()

,bjd_nm,road_nm,bunji,bld_nm,exclusive_area,land_area,cntrc_ym,cntrc_day,trade_price,floor,cnstr_year,dong,cntrc_year,price_per_area
0,인천광역시 연수구 선학동,넘말로47번길 37,393-7,"구룡마을(103,104)동",54.08,26.75,201301,2,9000,4,1995,선학동,2013,166.420118
1,인천광역시 연수구 옥련동,청룡로 40,348-160,한샘빌라(348-160),36.42,21.97,201301,17,5000,1,1991,옥련동,2013,137.287205
2,인천광역시 연수구 청학동,청학로12번길 29,535-4,금잔디빌라(535-4),35.70,27.20,201301,18,4000,1,1990,청학동,2013,112.044818
3,인천광역시 연수구 청학동,청학로6번길 85,550-15,백학맨션8차(550-15),53.80,29.94,201301,8,8300,1,1996,청학동,2013,154.275093
4,인천광역시 연수구 청학동,청학안로 12,519-5,산장타운(519-5),35.10,22.45,201301,4,7500,3,1995,청학동,2013,213.675214


In [20]:
villa_trade = villa_trade[(villa_trade['cntrc_year'] == 2018)
                          | (villa_trade['cntrc_year'] == 2019)
                          | (villa_trade['cntrc_year']==2020)
                          | (villa_trade['cntrc_year']==2021)]
villa_trade.reset_index(inplace=True)
villa_trade.drop('index', axis=1, inplace=True)
villa_trade.head()

,bjd_nm,road_nm,bunji,bld_nm,exclusive_area,land_area,cntrc_ym,cntrc_day,trade_price,floor,cnstr_year,dong,cntrc_year,price_per_area
0,인천광역시 연수구 동춘동,청능대로78번길 29-6,1116-6,동춘베스트빌,57.59,58.45,201801,3,16900,1,2009,동춘동,2018,293.453725
1,인천광역시 연수구 선학동,학나래로109번길 14,399-8,대창맨션,59.71,41.85,201801,23,15550,1,1997,선학동,2018,260.425389
2,인천광역시 연수구 선학동,학나래로 50,382,동민빌리지,73.75,42.94,201801,28,16400,3,2009,선학동,2018,222.372881
3,인천광역시 연수구 선학동,넘말로47번길 26,390-4,"아름빌라(1,2동)",39.93,25.01,201801,8,7300,3,1994,선학동,2018,182.819935
4,인천광역시 연수구 선학동,넘말로47번길 26,390-4,"아름빌라(1,2동)",39.93,25.01,201801,17,6800,1,1994,선학동,2018,170.298022


## 최종 데이터프레임

In [21]:
villa_trade_final = pd.DataFrame({'end_nm': villa_trade['dong'], 'base_date': villa_trade['cntrc_year'],
                                  'exclusive_area': villa_trade['exclusive_area'], 'price_per_area': villa_trade['price_per_area'].round(3),
                                   'cnstr_year': villa_trade['cnstr_year'].astype(int),
                                  'floor': villa_trade['floor']
                                   })

In [22]:
villa_trade_final['bld_type'] = '연립다세대'

In [23]:
villa_trade_final.head()

,end_nm,base_date,exclusive_area,price_per_area,cnstr_year,floor,bld_type
0,동춘동,2018,57.59,293.454,2009,1,연립다세대
1,선학동,2018,59.71,260.425,1997,1,연립다세대
2,선학동,2018,73.75,222.373,2009,3,연립다세대
3,선학동,2018,39.93,182.820,1994,3,연립다세대
4,선학동,2018,39.93,170.298,1994,1,연립다세대


# 전처리 - 18_연수구_오피스텔_매매_실거래가

In [24]:
officetel_trade.head()

,bjd_nm,road_nm,bunji,cmplx_nm,exclusive_area,cntrc_ym,cntrc_day,trade_price,floor,cnstr_year
0,인천광역시 연수구 송도동,컨벤시아대로42번길 20,15-10,송도 더 프라우 2,33.71,201301,14,12800,2,2012.0
1,인천광역시 연수구 송도동,컨벤시아대로42번길 20,15-10,송도 더 프라우 2,33.71,201301,14,13120,2,2012.0
2,인천광역시 연수구 송도동,해돋이로 107,4-1,송도 더샵 퍼스트월드,47.42,201301,2,15200,18,2009.0
3,인천광역시 연수구 송도동,해돋이로 107,4-1,송도 더샵 퍼스트월드,45.10,201301,14,15200,14,2009.0
4,인천광역시 연수구 송도동,해돋이로 107,4-1,송도 더샵 퍼스트월드,59.57,201301,28,20000,20,2009.0


In [25]:
officetel_trade.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5608 entries, 0 to 5607
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   bjd_nm          5608 non-null   object 
 1   road_nm         5608 non-null   object 
 2   bunji           5608 non-null   object 
 3   cmplx_nm        5608 non-null   object 
 4   exclusive_area  5608 non-null   float64
 5   cntrc_ym        5608 non-null   int64  
 6   cntrc_day       5608 non-null   int64  
 7   trade_price     5608 non-null   int64  
 8   floor           5608 non-null   int64  
 9   cnstr_year      5589 non-null   float64
dtypes: float64(2), int64(4), object(4)
memory usage: 438.2+ KB


In [26]:
officetel_trade[officetel_trade['cnstr_year'].isnull()]
# 해당 오피스텔은 2017년 입주 
# 출처: https://www.aptbong.com/view.php?bt=yenda&dn=311708 (593-10) https://www.aptbong.com/view.php?bt=yenda&dn=313757 (593-11))
# -> cnstr_year = 2017 채워넣기

,bjd_nm,road_nm,bunji,cmplx_nm,exclusive_area,cntrc_ym,cntrc_day,trade_price,floor,cnstr_year
1885,인천광역시 연수구 연수동,,593-10,이레하이니스,65.20,201707,10,20900,2,NaN
1886,인천광역시 연수구 연수동,,593-11,이레하이닉스,70.32,201707,15,21700,8,NaN
2257,인천광역시 연수구 연수동,,593-10,이레하이니스,56.45,201905,22,14900,10,NaN
2299,인천광역시 연수구 연수동,,593-10,이레하이니스,56.45,201906,25,15800,10,NaN
2356,인천광역시 연수구 연수동,,593-10,이레하이니스,57.60,201911,15,17600,6,NaN
2472,인천광역시 연수구 연수동,,593-11,이레하이닉스,70.32,202004,17,17000,8,NaN
2580,인천광역시 연수구 연수동,,593-10,이레하이니스,65.20,202006,26,20500,7,NaN
2658,인천광역시 연수구 연수동,,593-10,이레하이니스,57.60,202008,14,18100,3,NaN
3101,인천광역시 연수구 연수동,,593-10,이레하이니스,65.20,202101,3,18900,8,NaN
3709,인천광역시 연수구 연수동,,593-10,이레하이니스,56.45,202104,7,17600,7,NaN


In [27]:
officetel_trade = officetel_trade.fillna(2017)

In [28]:
dong = []
for i in officetel_trade['bjd_nm']:
    dong.append(i[-3:])
officetel_trade['dong'] = dong

cntrc = []
for i in officetel_trade['cntrc_ym']:
    cntrc.append(str(i)[0:4])

cntrc_year = []
for year in cntrc:
    cntrc_year.append(int(year))
officetel_trade['cntrc_year'] = cntrc_year

price_per_area = [] # 제곱미터당 가격
for i in range(len(officetel_trade)):
    price_per_area.append( officetel_trade['trade_price'][i] / officetel_trade['exclusive_area'][i] )
officetel_trade['price_per_area'] = price_per_area

In [29]:
officetel_trade.head()

,bjd_nm,road_nm,bunji,cmplx_nm,exclusive_area,cntrc_ym,cntrc_day,trade_price,floor,cnstr_year,dong,cntrc_year,price_per_area
0,인천광역시 연수구 송도동,컨벤시아대로42번길 20,15-10,송도 더 프라우 2,33.71,201301,14,12800,2,2012.0,송도동,2013,379.709285
1,인천광역시 연수구 송도동,컨벤시아대로42번길 20,15-10,송도 더 프라우 2,33.71,201301,14,13120,2,2012.0,송도동,2013,389.202017
2,인천광역시 연수구 송도동,해돋이로 107,4-1,송도 더샵 퍼스트월드,47.42,201301,2,15200,18,2009.0,송도동,2013,320.539857
3,인천광역시 연수구 송도동,해돋이로 107,4-1,송도 더샵 퍼스트월드,45.10,201301,14,15200,14,2009.0,송도동,2013,337.028825
4,인천광역시 연수구 송도동,해돋이로 107,4-1,송도 더샵 퍼스트월드,59.57,201301,28,20000,20,2009.0,송도동,2013,335.739466


In [30]:
officetel_trade = officetel_trade[(officetel_trade['cntrc_year'] == 2018)
                          | (officetel_trade['cntrc_year'] == 2019)
                          | (officetel_trade['cntrc_year']==2020)
                          | (officetel_trade['cntrc_year']==2021)]
                          
officetel_trade.reset_index(inplace=True)
officetel_trade.drop('index', axis=1, inplace=True)
officetel_trade.head()

,bjd_nm,road_nm,bunji,cmplx_nm,exclusive_area,cntrc_ym,cntrc_day,trade_price,floor,cnstr_year,dong,cntrc_year,price_per_area
0,인천광역시 연수구 송도동,송도과학로27번길 55,149,롯데캐슬 캠퍼스타운,34.68,201801,3,15600,9,2016.0,송도동,2018,449.826990
1,인천광역시 연수구 송도동,송도과학로27번길 55,149,롯데캐슬 캠퍼스타운,34.68,201801,12,16450,8,2016.0,송도동,2018,474.336794
2,인천광역시 연수구 송도동,송도과학로27번길 55,149,롯데캐슬 캠퍼스타운,34.55,201801,15,15800,4,2016.0,송도동,2018,457.308249
3,인천광역시 연수구 송도동,송도과학로27번길 55,149,롯데캐슬 캠퍼스타운,33.34,201801,20,16500,15,2016.0,송도동,2018,494.901020
4,인천광역시 연수구 송도동,송도과학로27번길 55,149,롯데캐슬 캠퍼스타운,34.68,201801,23,16100,9,2016.0,송도동,2018,464.244521


## 최종 데이터프레임

In [31]:
officetel_trade_final = pd.DataFrame({'end_nm': officetel_trade['dong'], 'base_date': officetel_trade['cntrc_year'],
                                  'exclusive_area': officetel_trade['exclusive_area'], 'price_per_area': officetel_trade['price_per_area'].round(3),
                                   'cnstr_year': officetel_trade['cnstr_year'].astype(int),
                                 'floor': officetel_trade['floor']
                                   })

In [32]:
officetel_trade_final['bld_type'] = '오피스텔'

In [33]:
officetel_trade_final.head()

,end_nm,base_date,exclusive_area,price_per_area,cnstr_year,floor,bld_type
0,송도동,2018,34.68,449.827,2016,9,오피스텔
1,송도동,2018,34.68,474.337,2016,8,오피스텔
2,송도동,2018,34.55,457.308,2016,4,오피스텔
3,송도동,2018,33.34,494.901,2016,15,오피스텔
4,송도동,2018,34.68,464.245,2016,9,오피스텔


# 매매 최종 데이터프레임

In [34]:
trade_final = pd.concat([apart_trade_final, villa_trade_final, officetel_trade_final], ignore_index=True)
trade_final.head()

,end_nm,base_date,exclusive_area,price_per_area,cnstr_year,floor,bld_type
0,동춘동,2018,84.945,282.536,1994,5,아파트
1,동춘동,2018,73.660,323.106,1994,2,아파트
2,동춘동,2018,84.945,297.840,1994,3,아파트
3,동춘동,2018,84.945,282.536,1994,5,아파트
4,동춘동,2018,84.945,291.954,1994,1,아파트


In [35]:
trade_final 

,end_nm,base_date,exclusive_area,price_per_area,cnstr_year,floor,bld_type
0,동춘동,2018,84.945,282.536,1994,5,아파트
1,동춘동,2018,73.660,323.106,1994,2,아파트
2,동춘동,2018,84.945,297.840,1994,3,아파트
3,동춘동,2018,84.945,282.536,1994,5,아파트
4,동춘동,2018,84.945,291.954,1994,1,아파트
...,...,...,...,...,...,...,...
32386,송도동,2021,84.030,595.026,2020,8,오피스텔
32387,연수동,2021,35.000,202.857,2003,6,오피스텔
32388,연수동,2021,26.550,150.659,2002,8,오피스텔
32389,옥련동,2021,56.760,241.367,2014,2,오피스텔


In [36]:
trade_final.to_csv('최종_공동주택_매매_실거래가.csv')

# 전처리 - 19_연수구_아파트_전월세_실거래가

In [37]:
apart_rent.head()

,bjd_nm,road_nm,bunji,cmplx_nm,rent_type,exclusive_area,cntrc_ym,cntrc_day,deposit,m_rent,floor,cnstr_year
0,인천광역시 연수구 동춘동,먼우금로83번길 12,938,건영,전세,84.945,201301,19,15500,0,4,1994.0
1,인천광역시 연수구 동춘동,먼우금로83번길 12,938,건영,월세,73.660,201301,23,1500,65,2,1994.0
2,인천광역시 연수구 동춘동,먼우금로83번길 12,938,건영,전세,84.945,201301,25,13000,0,5,1994.0
3,인천광역시 연수구 동춘동,먼우금로83번길 12,938,건영,전세,84.945,201301,28,14500,0,2,1994.0
4,인천광역시 연수구 동춘동,먼우금로 69,941,대동,전세,75.255,201301,9,12000,0,1,1994.0


In [38]:
apart_rent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79075 entries, 0 to 79074
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   bjd_nm          79075 non-null  object 
 1   road_nm         79075 non-null  object 
 2   bunji           79075 non-null  object 
 3   cmplx_nm        79075 non-null  object 
 4   rent_type       79075 non-null  object 
 5   exclusive_area  79075 non-null  float64
 6   cntrc_ym        79075 non-null  int64  
 7   cntrc_day       79075 non-null  int64  
 8   deposit         79075 non-null  int64  
 9   m_rent          79075 non-null  int64  
 10  floor           79075 non-null  int64  
 11  cnstr_year      79043 non-null  float64
dtypes: float64(2), int64(5), object(5)
memory usage: 7.2+ MB


In [39]:
apart_rent[apart_rent['cnstr_year'].isnull()]
# 더샵송도프라임뷰25BL: 2021년 입주 (출처: https://www.zigbang.com/home/apt/danjis/79581)
# -> cnstr_year = 2021 채워넣기

,bjd_nm,road_nm,bunji,cmplx_nm,rent_type,exclusive_area,cntrc_ym,cntrc_day,deposit,m_rent,floor,cnstr_year
68509,인천광역시 연수구 송도동,인천타워대로231번길 117,114,더샵송도프라임뷰25BL,전세,121.18,202107,24,87000,0,2,NaN
69389,인천광역시 연수구 송도동,인천타워대로231번길 117,114,더샵송도프라임뷰25BL,월세,144.92,202108,19,65000,300,15,NaN
69390,인천광역시 연수구 송도동,인천타워대로231번길 117,114,더샵송도프라임뷰25BL,전세,121.18,202108,24,105000,0,14,NaN
70271,인천광역시 연수구 송도동,인천타워대로231번길 117,114,더샵송도프라임뷰25BL,전세,121.18,202109,10,115000,0,10,NaN
71028,인천광역시 연수구 송도동,인천타워대로231번길 117,114,더샵송도프라임뷰25BL,전세,107.60,202110,22,85000,0,5,NaN
71029,인천광역시 연수구 송도동,인천타워대로231번길 117,114,더샵송도프라임뷰25BL,전세,107.60,202110,26,85000,0,8,NaN
71912,인천광역시 연수구 송도동,인천타워대로231번길 117,114,더샵송도프라임뷰25BL,월세,121.18,202111,10,60000,150,8,NaN
71913,인천광역시 연수구 송도동,인천타워대로231번길 117,114,더샵송도프라임뷰25BL,전세,107.60,202111,22,80000,0,8,NaN
71914,인천광역시 연수구 송도동,인천타워대로231번길 117,114,더샵송도프라임뷰25BL,전세,107.60,202111,27,80000,0,9,NaN
71915,인천광역시 연수구 송도동,인천타워대로231번길 117,114,더샵송도프라임뷰25BL,전세,107.60,202111,27,80000,0,6,NaN


In [40]:
apart_rent = apart_rent.fillna(2021)

In [41]:
dong = []
for i in apart_rent['bjd_nm']:
    dong.append(i[-3:])
apart_rent['dong'] = dong

cntrc = []
for i in apart_rent['cntrc_ym']:
    cntrc.append(str(i)[0:4])

cntrc_year = []
for year in cntrc:
    cntrc_year.append(int(year))
apart_rent['cntrc_year'] = cntrc_year

In [42]:
apart_rent['type'] = '아파트'

In [43]:
apart_rent.head()

,bjd_nm,road_nm,bunji,cmplx_nm,rent_type,exclusive_area,cntrc_ym,cntrc_day,deposit,m_rent,floor,cnstr_year,dong,cntrc_year,type
0,인천광역시 연수구 동춘동,먼우금로83번길 12,938,건영,전세,84.945,201301,19,15500,0,4,1994.0,동춘동,2013,아파트
1,인천광역시 연수구 동춘동,먼우금로83번길 12,938,건영,월세,73.660,201301,23,1500,65,2,1994.0,동춘동,2013,아파트
2,인천광역시 연수구 동춘동,먼우금로83번길 12,938,건영,전세,84.945,201301,25,13000,0,5,1994.0,동춘동,2013,아파트
3,인천광역시 연수구 동춘동,먼우금로83번길 12,938,건영,전세,84.945,201301,28,14500,0,2,1994.0,동춘동,2013,아파트
4,인천광역시 연수구 동춘동,먼우금로 69,941,대동,전세,75.255,201301,9,12000,0,1,1994.0,동춘동,2013,아파트


In [44]:
apart_rent = apart_rent[(apart_rent['cntrc_year'] == 2018)
                          | (apart_rent['cntrc_year'] == 2019)
                          | (apart_rent['cntrc_year']==2020)
                          | (apart_rent['cntrc_year']==2021)]
                          
apart_rent.reset_index(inplace=True)
apart_rent.drop('index', axis=1, inplace=True)
apart_rent.head()

,bjd_nm,road_nm,bunji,cmplx_nm,rent_type,exclusive_area,cntrc_ym,cntrc_day,deposit,m_rent,floor,cnstr_year,dong,cntrc_year,type
0,인천광역시 연수구 동춘동,먼우금로83번길 12,938,건영,전세,84.945,201801,3,21000,0,3,1994.0,동춘동,2018,아파트
1,인천광역시 연수구 동춘동,먼우금로83번길 12,938,건영,월세,84.945,201801,6,2000,70,2,1994.0,동춘동,2018,아파트
2,인천광역시 연수구 동춘동,먼우금로83번길 12,938,건영,전세,84.945,201801,12,20000,0,4,1994.0,동춘동,2018,아파트
3,인천광역시 연수구 동춘동,먼우금로83번길 12,938,건영,전세,84.945,201801,13,20000,0,2,1994.0,동춘동,2018,아파트
4,인천광역시 연수구 동춘동,먼우금로 69,941,대동,전세,84.585,201801,6,16000,0,2,1994.0,동춘동,2018,아파트


## 전월세 분리

In [45]:
apart_rent_long = apart_rent[apart_rent['rent_type'] == '전세'].reset_index()
apart_rent_month = apart_rent[apart_rent['rent_type'] == '월세'].reset_index()

## 전세

In [46]:
apart_rent_long.head()

,index,bjd_nm,road_nm,bunji,cmplx_nm,rent_type,exclusive_area,cntrc_ym,cntrc_day,deposit,m_rent,floor,cnstr_year,dong,cntrc_year,type
0,0,인천광역시 연수구 동춘동,먼우금로83번길 12,938,건영,전세,84.945,201801,3,21000,0,3,1994.0,동춘동,2018,아파트
1,2,인천광역시 연수구 동춘동,먼우금로83번길 12,938,건영,전세,84.945,201801,12,20000,0,4,1994.0,동춘동,2018,아파트
2,3,인천광역시 연수구 동춘동,먼우금로83번길 12,938,건영,전세,84.945,201801,13,20000,0,2,1994.0,동춘동,2018,아파트
3,4,인천광역시 연수구 동춘동,먼우금로 69,941,대동,전세,84.585,201801,6,16000,0,2,1994.0,동춘동,2018,아파트
4,5,인천광역시 연수구 동춘동,먼우금로 69,941,대동,전세,75.255,201801,11,20000,0,6,1994.0,동춘동,2018,아파트


In [47]:
deposit_per_area = [] # 제곱미터당 보증금
for i in range(len(apart_rent_long)):
    deposit_per_area.append(apart_rent_long['deposit'][i] / apart_rent_long['exclusive_area'][i])

apart_rent_long['deposit_per_area'] = deposit_per_area

In [48]:
apart_rent_long.head()

,index,bjd_nm,road_nm,bunji,cmplx_nm,rent_type,exclusive_area,cntrc_ym,cntrc_day,deposit,m_rent,floor,cnstr_year,dong,cntrc_year,type,deposit_per_area
0,0,인천광역시 연수구 동춘동,먼우금로83번길 12,938,건영,전세,84.945,201801,3,21000,0,3,1994.0,동춘동,2018,아파트,247.218789
1,2,인천광역시 연수구 동춘동,먼우금로83번길 12,938,건영,전세,84.945,201801,12,20000,0,4,1994.0,동춘동,2018,아파트,235.446465
2,3,인천광역시 연수구 동춘동,먼우금로83번길 12,938,건영,전세,84.945,201801,13,20000,0,2,1994.0,동춘동,2018,아파트,235.446465
3,4,인천광역시 연수구 동춘동,먼우금로 69,941,대동,전세,84.585,201801,6,16000,0,2,1994.0,동춘동,2018,아파트,189.158834
4,5,인천광역시 연수구 동춘동,먼우금로 69,941,대동,전세,75.255,201801,11,20000,0,6,1994.0,동춘동,2018,아파트,265.763072


### 최종 데이터프레임

In [49]:
apart_rent_long_final = pd.DataFrame({'end_nm': apart_rent_long['dong'], 'base_date': apart_rent_long['cntrc_year'], 
                                      'deposit_per_area': apart_rent_long['deposit_per_area'],
                                      'exclusive_area': apart_rent_long['exclusive_area'],
                                       'cnstr_year': apart_rent_long['cnstr_year'].astype(int), 
                                 'floor': apart_rent_long['floor'], 'type': apart_rent_long['type']})

In [50]:
apart_rent_long_final.head()

,end_nm,base_date,deposit_per_area,exclusive_area,cnstr_year,floor,type
0,동춘동,2018,247.218789,84.945,1994,3,아파트
1,동춘동,2018,235.446465,84.945,1994,4,아파트
2,동춘동,2018,235.446465,84.945,1994,2,아파트
3,동춘동,2018,189.158834,84.585,1994,2,아파트
4,동춘동,2018,265.763072,75.255,1994,6,아파트


## 월세

In [51]:
apart_rent_month.head()

,index,bjd_nm,road_nm,bunji,cmplx_nm,rent_type,exclusive_area,cntrc_ym,cntrc_day,deposit,m_rent,floor,cnstr_year,dong,cntrc_year,type
0,1,인천광역시 연수구 동춘동,먼우금로83번길 12,938,건영,월세,84.945,201801,6,2000,70,2,1994.0,동춘동,2018,아파트
1,7,인천광역시 연수구 동춘동,먼우금로83번길 49,940,대림3,월세,53.010,201801,2,2000,40,6,1993.0,동춘동,2018,아파트
2,13,인천광역시 연수구 동춘동,먼우금로 19,943,동남,월세,52.140,201801,8,500,15,7,1994.0,동춘동,2018,아파트
3,14,인천광역시 연수구 동춘동,먼우금로 19,943,동남,월세,52.140,201801,9,1000,57,9,1994.0,동춘동,2018,아파트
4,18,인천광역시 연수구 동춘동,먼우금로 19,943,동남,월세,52.140,201801,13,2000,60,10,1994.0,동춘동,2018,아파트


### 최종 데이터프레임

In [52]:
apart_rent_month_final = pd.DataFrame({'end_nm': apart_rent_month['dong'], 'base_date': apart_rent_month['cntrc_year'],
                                      'deposit': apart_rent_month['deposit'], 'm_rent': apart_rent_month['m_rent'],
                                       'exclusive_area': apart_rent_month['exclusive_area'],
                                       'cnstr_year': apart_rent_month['cnstr_year'].astype(int),
                                 'floor': apart_rent_month['floor'], 'type': apart_rent_month['type']})

In [53]:
apart_rent_month_final.head()

,end_nm,base_date,deposit,m_rent,exclusive_area,cnstr_year,floor,type
0,동춘동,2018,2000,70,84.945,1994,2,아파트
1,동춘동,2018,2000,40,53.010,1993,6,아파트
2,동춘동,2018,500,15,52.140,1994,7,아파트
3,동춘동,2018,1000,57,52.140,1994,9,아파트
4,동춘동,2018,2000,60,52.140,1994,10,아파트


# 전처리 - 20_연수구_연립다세대_전월세_실거래가

In [54]:
villa_rent.head()

,bjd_nm,road_nm,bunji,bld_nm,rent_type,exclusive_area,cntrc_ym,cntrc_day,deposit,m_rent,floor,cnstr_year
0,인천광역시 연수구 동춘동,솔밭로 64,810,(810-0),월세,44.270,201301,4,1500,34,2.0,2009
1,인천광역시 연수구 선학동,학나래로117번길 14,400-2,남가빌리지(103동),월세,52.340,201301,24,500,45,2.0,1995
2,인천광역시 연수구 선학동,학나래로117번길 8,400-6,선학마을1동,전세,35.890,201301,8,4500,0,3.0,1996
3,인천광역시 연수구 선학동,넘말로47번길 16-1,390,"아름빌라(10,11동)",전세,36.845,201301,22,4500,0,3.0,1995
4,인천광역시 연수구 선학동,넘말로47번길 22-1,390-3,"아름빌라(3,5동)",전세,39.930,201301,30,4000,0,1.0,1995


In [55]:
villa_rent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3229 entries, 0 to 3228
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   bjd_nm          3229 non-null   object 
 1   road_nm         3229 non-null   object 
 2   bunji           3229 non-null   object 
 3   bld_nm          3229 non-null   object 
 4   rent_type       3229 non-null   object 
 5   exclusive_area  3229 non-null   float64
 6   cntrc_ym        3229 non-null   int64  
 7   cntrc_day       3229 non-null   int64  
 8   deposit         3229 non-null   int64  
 9   m_rent          3229 non-null   int64  
 10  floor           3174 non-null   float64
 11  cnstr_year      3229 non-null   int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 302.8+ KB


In [56]:
villa_rent[villa_rent['floor'].isnull()]
# null값을 구할 방법이 없고, null값의 수가 많지 않음. -> 삭제처리

,bjd_nm,road_nm,bunji,bld_nm,rent_type,exclusive_area,cntrc_ym,cntrc_day,deposit,m_rent,floor,cnstr_year
109,인천광역시 연수구 옥련동,한나루로86번길 53,494-10,지오타운,전세,40.280,201303,20,2400,0,NaN,2013
192,인천광역시 연수구 옥련동,독배로82번길 33-16,158-1,미래하이츠,전세,80.990,201305,24,9000,0,NaN,2002
203,인천광역시 연수구 옥련동,한나루로86번길 53,494-10,지오타운,전세,65.000,201305,7,2500,0,NaN,2013
231,인천광역시 연수구 선학동,넘말로47번길,390,"아름빌라(10,11동)",전세,39.930,201306,25,5000,0,NaN,1995
248,인천광역시 연수구 옥련동,청량로164번길 45,405-24,옥련하이츠빌라,월세,137.790,201306,7,2500,60,NaN,1995
284,인천광역시 연수구 옥련동,청룡로 27,348-173,우림빌라(348-173),전세,36.360,201307,30,2500,0,NaN,1990
351,인천광역시 연수구 동춘동,봉재산로 92,344,산골마을103동,월세,43.270,201309,3,500,35,NaN,2002
357,인천광역시 연수구 옥련동,독배로90번길 10,170-14,SweetForestB동,전세,84.490,201309,5,11000,0,NaN,2009
366,인천광역시 연수구 옥련동,한나루로171번길 5,352-5,정원스위트빌2,월세,70.100,201309,6,2500,50,NaN,2013
412,인천광역시 연수구 옥련동,한나루로171번길 5,352-5,정원스위트빌2,월세,65.800,201310,1,2000,60,NaN,2013


In [57]:
villa_rent = villa_rent.dropna()

In [58]:
dong = []
for i in villa_rent['bjd_nm']:
    dong.append(i[-3:])
villa_rent['dong'] = dong

cntrc = []
for i in villa_rent['cntrc_ym']:
    cntrc.append(str(i)[0:4])

cntrc_year = []
for year in cntrc:
    cntrc_year.append(int(year))
villa_rent['cntrc_year'] = cntrc_year

In [59]:
villa_rent['type'] = '연립다세대'

In [60]:
villa_rent.head()

,bjd_nm,road_nm,bunji,bld_nm,rent_type,exclusive_area,cntrc_ym,cntrc_day,deposit,m_rent,floor,cnstr_year,dong,cntrc_year,type
0,인천광역시 연수구 동춘동,솔밭로 64,810,(810-0),월세,44.270,201301,4,1500,34,2.0,2009,동춘동,2013,연립다세대
1,인천광역시 연수구 선학동,학나래로117번길 14,400-2,남가빌리지(103동),월세,52.340,201301,24,500,45,2.0,1995,선학동,2013,연립다세대
2,인천광역시 연수구 선학동,학나래로117번길 8,400-6,선학마을1동,전세,35.890,201301,8,4500,0,3.0,1996,선학동,2013,연립다세대
3,인천광역시 연수구 선학동,넘말로47번길 16-1,390,"아름빌라(10,11동)",전세,36.845,201301,22,4500,0,3.0,1995,선학동,2013,연립다세대
4,인천광역시 연수구 선학동,넘말로47번길 22-1,390-3,"아름빌라(3,5동)",전세,39.930,201301,30,4000,0,1.0,1995,선학동,2013,연립다세대


In [61]:
villa_rent = villa_rent[(villa_rent['cntrc_year'] == 2018)
                          | (villa_rent['cntrc_year'] == 2019)
                          | (villa_rent['cntrc_year']==2020)
                          | (villa_rent['cntrc_year']==2021)]
                          
villa_rent.reset_index(inplace=True)
villa_rent.drop('index', axis=1, inplace=True)
villa_rent.head()

,bjd_nm,road_nm,bunji,bld_nm,rent_type,exclusive_area,cntrc_ym,cntrc_day,deposit,m_rent,floor,cnstr_year,dong,cntrc_year,type
0,인천광역시 연수구 선학동,넘말로55번길 36,387-3,전원그린빌라,월세,62.71,201801,4,2000,50,3.0,1997,선학동,2018,연립다세대
1,인천광역시 연수구 옥련동,대암로14번길 25,562-2,나우아트빌,전세,36.80,201801,3,6000,0,1.0,2007,옥련동,2018,연립다세대
2,인천광역시 연수구 옥련동,능허대로 121-5,194-31,대림빌라,월세,50.60,201801,30,1000,50,1.0,1992,옥련동,2018,연립다세대
3,인천광역시 연수구 옥련동,한나루로158번길 31,348-141,벽산,전세,56.58,201801,2,8500,0,1.0,1991,옥련동,2018,연립다세대
4,인천광역시 연수구 옥련동,한나루로158번길 31,348-141,벽산,전세,50.01,201801,31,11000,0,3.0,1991,옥련동,2018,연립다세대


## 전월세 분리

In [62]:
villa_rent_long = villa_rent[villa_rent['rent_type'] == '전세'].reset_index()
villa_rent_month = villa_rent[villa_rent['rent_type'] == '월세'].reset_index()

## 전세

In [63]:
villa_rent_long.head()

,index,bjd_nm,road_nm,bunji,bld_nm,rent_type,exclusive_area,cntrc_ym,cntrc_day,deposit,m_rent,floor,cnstr_year,dong,cntrc_year,type
0,1,인천광역시 연수구 옥련동,대암로14번길 25,562-2,나우아트빌,전세,36.80,201801,3,6000,0,1.0,2007,옥련동,2018,연립다세대
1,3,인천광역시 연수구 옥련동,한나루로158번길 31,348-141,벽산,전세,56.58,201801,2,8500,0,1.0,1991,옥련동,2018,연립다세대
2,4,인천광역시 연수구 옥련동,한나루로158번길 31,348-141,벽산,전세,50.01,201801,31,11000,0,3.0,1991,옥련동,2018,연립다세대
3,5,인천광역시 연수구 옥련동,청량로164번길 11,405-10,부영하이츠빌2차(405-10),전세,38.67,201801,4,6500,0,2.0,2001,옥련동,2018,연립다세대
4,6,인천광역시 연수구 옥련동,청량로164번길 11,405-10,부영하이츠빌2차(405-10),전세,37.83,201801,30,6500,0,1.0,2001,옥련동,2018,연립다세대


In [64]:
deposit_per_area = [] # 제곱미터당 보증금
for i in range(len(villa_rent_long)):
    deposit_per_area.append(villa_rent_long['deposit'][i] / villa_rent_long['exclusive_area'][i])

villa_rent_long['deposit_per_area'] = deposit_per_area

In [65]:
villa_rent_long.head()

,index,bjd_nm,road_nm,bunji,bld_nm,rent_type,exclusive_area,cntrc_ym,cntrc_day,deposit,m_rent,floor,cnstr_year,dong,cntrc_year,type,deposit_per_area
0,1,인천광역시 연수구 옥련동,대암로14번길 25,562-2,나우아트빌,전세,36.80,201801,3,6000,0,1.0,2007,옥련동,2018,연립다세대,163.043478
1,3,인천광역시 연수구 옥련동,한나루로158번길 31,348-141,벽산,전세,56.58,201801,2,8500,0,1.0,1991,옥련동,2018,연립다세대,150.229763
2,4,인천광역시 연수구 옥련동,한나루로158번길 31,348-141,벽산,전세,50.01,201801,31,11000,0,3.0,1991,옥련동,2018,연립다세대,219.956009
3,5,인천광역시 연수구 옥련동,청량로164번길 11,405-10,부영하이츠빌2차(405-10),전세,38.67,201801,4,6500,0,2.0,2001,옥련동,2018,연립다세대,168.088958
4,6,인천광역시 연수구 옥련동,청량로164번길 11,405-10,부영하이츠빌2차(405-10),전세,37.83,201801,30,6500,0,1.0,2001,옥련동,2018,연립다세대,171.821306


### 최종 데이터프레임

In [66]:
villa_rent_long_final = pd.DataFrame({'end_nm': villa_rent_long['dong'], 'base_date': villa_rent_long['cntrc_year'],
                                      'deposit_per_area': villa_rent_long['deposit_per_area'], 'exclusive_area': villa_rent_long['exclusive_area'],
                                       'cnstr_year': villa_rent_long['cnstr_year'].astype(int),
                                 'floor': villa_rent_long['floor'], 'type': villa_rent_long['type']})

In [67]:
villa_rent_long_final.head()

,end_nm,base_date,deposit_per_area,exclusive_area,cnstr_year,floor,type
0,옥련동,2018,163.043478,36.80,2007,1.0,연립다세대
1,옥련동,2018,150.229763,56.58,1991,1.0,연립다세대
2,옥련동,2018,219.956009,50.01,1991,3.0,연립다세대
3,옥련동,2018,168.088958,38.67,2001,2.0,연립다세대
4,옥련동,2018,171.821306,37.83,2001,1.0,연립다세대


## 월세

In [68]:
villa_rent_month.head()

,index,bjd_nm,road_nm,bunji,bld_nm,rent_type,exclusive_area,cntrc_ym,cntrc_day,deposit,m_rent,floor,cnstr_year,dong,cntrc_year,type
0,0,인천광역시 연수구 선학동,넘말로55번길 36,387-3,전원그린빌라,월세,62.71,201801,4,2000,50,3.0,1997,선학동,2018,연립다세대
1,2,인천광역시 연수구 옥련동,능허대로 121-5,194-31,대림빌라,월세,50.60,201801,30,1000,50,1.0,1992,옥련동,2018,연립다세대
2,12,인천광역시 연수구 청학동,용담로85번길 19,497-3,룩소르빌딩,월세,21.70,201801,2,2000,25,4.0,2012,청학동,2018,연립다세대
3,20,인천광역시 연수구 옥련동,옥련로99번길 3,248-20,양지빌라(248-20),월세,34.14,201802,9,500,40,2.0,1991,옥련동,2018,연립다세대
4,24,인천광역시 연수구 옥련동,독배로 85-1,173-26,플로리체(102동),월세,55.27,201802,13,1000,90,5.0,2016,옥련동,2018,연립다세대


### 최종 데이터프레임

In [69]:
villa_rent_month_final = pd.DataFrame({'end_nm': villa_rent_month['dong'], 'base_date': villa_rent_month['cntrc_year'],
                                      'deposit': villa_rent_month['deposit'], 'm_rent': villa_rent_month['m_rent'],
                                       'exclusive_area': villa_rent_month['exclusive_area'],
                                       'cnstr_year': villa_rent_month['cnstr_year'].astype(int),
                                 'floor': villa_rent_month['floor'], 'type': villa_rent_month['type']})

In [70]:
villa_rent_month_final.head()

,end_nm,base_date,deposit,m_rent,exclusive_area,cnstr_year,floor,type
0,선학동,2018,2000,50,62.71,1997,3.0,연립다세대
1,옥련동,2018,1000,50,50.60,1992,1.0,연립다세대
2,청학동,2018,2000,25,21.70,2012,4.0,연립다세대
3,옥련동,2018,500,40,34.14,1991,2.0,연립다세대
4,옥련동,2018,1000,90,55.27,2016,5.0,연립다세대


# 전처리 - 22_연수구_오피스텔_전월세_실거래가

In [71]:
officetel_rent.head()

,bjd_nm,road_nm,bunji,cmplx_nm,rent_type,exclusive_area,cntrc_ym,cntrc_day,deposit,m_rent,floor,cnstr_year
0,인천광역시 연수구 송도동,해돋이로 107,4-1,송도 더샵 퍼스트월드,전세,46.57,201301,4,7200,0,22,2009.0
1,인천광역시 연수구 송도동,해돋이로 107,4-1,송도 더샵 퍼스트월드,전세,45.10,201301,7,8000,0,19,2009.0
2,인천광역시 연수구 송도동,해돋이로 107,4-1,송도 더샵 퍼스트월드,전세,59.57,201301,19,8000,0,4,2009.0
3,인천광역시 연수구 송도동,해돋이로 107,4-1,송도 더샵 퍼스트월드,전세,51.73,201301,23,7000,0,6,2009.0
4,인천광역시 연수구 송도동,해돋이로 107,4-1,송도 더샵 퍼스트월드,전세,47.42,201301,29,8500,0,16,2009.0


In [72]:
officetel_rent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16060 entries, 0 to 16059
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   bjd_nm          16060 non-null  object 
 1   road_nm         16060 non-null  object 
 2   bunji           16060 non-null  object 
 3   cmplx_nm        16060 non-null  object 
 4   rent_type       16060 non-null  object 
 5   exclusive_area  16060 non-null  float64
 6   cntrc_ym        16060 non-null  int64  
 7   cntrc_day       16060 non-null  int64  
 8   deposit         16060 non-null  int64  
 9   m_rent          16060 non-null  int64  
 10  floor           16060 non-null  int64  
 11  cnstr_year      15992 non-null  float64
dtypes: float64(2), int64(5), object(5)
memory usage: 1.5+ MB


In [73]:
officetel_rent[officetel_rent['cnstr_year'].isnull()]
# 해당 오피스텔은 2017년 입주 
# 출처: https://www.aptbong.com/view.php?bt=yenda&dn=311708 (593-10) https://www.aptbong.com/view.php?bt=yenda&dn=313757 (593-11))
# -> cnstr_year = 2017 채워넣기

,bjd_nm,road_nm,bunji,cmplx_nm,rent_type,exclusive_area,cntrc_ym,cntrc_day,deposit,m_rent,floor,cnstr_year
3675,인천광역시 연수구 연수동,,593-10,이레하이니스,전세,65.20,201704,25,15000,0,2,NaN
3676,인천광역시 연수구 연수동,,593-11,이레하이닉스,월세,70.32,201704,25,3000,80,5,NaN
3815,인천광역시 연수구 연수동,,593-10,이레하이니스,월세,65.20,201705,3,5000,60,8,NaN
4129,인천광역시 연수구 연수동,,593-10,이레하이니스,월세,56.45,201707,17,1000,60,5,NaN
4130,인천광역시 연수구 연수동,,593-11,이레하이닉스,월세,63.70,201707,7,1000,70,4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
13205,인천광역시 연수구 연수동,,593-10,이레하이니스,전세,56.45,202105,22,16500,0,10,NaN
13431,인천광역시 연수구 연수동,,593-11,이레하이닉스,월세,63.70,202106,30,1000,70,8,NaN
13774,인천광역시 연수구 연수동,,593-10,이레하이니스,월세,56.45,202107,8,1000,60,5,NaN
13775,인천광역시 연수구 연수동,,593-11,이레하이닉스,월세,63.70,202107,11,3500,40,3,NaN


In [74]:
officetel_rent = officetel_rent.fillna(2017)

In [75]:
dong = []
for i in officetel_rent['bjd_nm']:
    dong.append(i[-3:])
officetel_rent['dong'] = dong

cntrc = []
for i in officetel_rent['cntrc_ym']:
    cntrc.append(str(i)[0:4])

cntrc_year = []
for year in cntrc:
    cntrc_year.append(int(year))
officetel_rent['cntrc_year'] = cntrc_year

In [76]:
officetel_rent['type'] = '오피스텔'

In [77]:
officetel_rent.head()

,bjd_nm,road_nm,bunji,cmplx_nm,rent_type,exclusive_area,cntrc_ym,cntrc_day,deposit,m_rent,floor,cnstr_year,dong,cntrc_year,type
0,인천광역시 연수구 송도동,해돋이로 107,4-1,송도 더샵 퍼스트월드,전세,46.57,201301,4,7200,0,22,2009.0,송도동,2013,오피스텔
1,인천광역시 연수구 송도동,해돋이로 107,4-1,송도 더샵 퍼스트월드,전세,45.10,201301,7,8000,0,19,2009.0,송도동,2013,오피스텔
2,인천광역시 연수구 송도동,해돋이로 107,4-1,송도 더샵 퍼스트월드,전세,59.57,201301,19,8000,0,4,2009.0,송도동,2013,오피스텔
3,인천광역시 연수구 송도동,해돋이로 107,4-1,송도 더샵 퍼스트월드,전세,51.73,201301,23,7000,0,6,2009.0,송도동,2013,오피스텔
4,인천광역시 연수구 송도동,해돋이로 107,4-1,송도 더샵 퍼스트월드,전세,47.42,201301,29,8500,0,16,2009.0,송도동,2013,오피스텔


In [78]:
officetel_rent = officetel_rent[(officetel_rent['cntrc_year'] == 2018)
                          | (officetel_rent['cntrc_year'] == 2019)
                          | (officetel_rent['cntrc_year']==2020)
                          | (officetel_rent['cntrc_year']==2021)]
                          
officetel_rent.reset_index(inplace=True)
officetel_rent.drop('index', axis=1, inplace=True)
officetel_rent.head()

,bjd_nm,road_nm,bunji,cmplx_nm,rent_type,exclusive_area,cntrc_ym,cntrc_day,deposit,m_rent,floor,cnstr_year,dong,cntrc_year,type
0,인천광역시 연수구 송도동,송도문화로28번길 28,190-2,글로벌 캠퍼스 푸르지오,전세,49.32,201801,2,17500,0,5,2013.0,송도동,2018,오피스텔
1,인천광역시 연수구 송도동,송도문화로28번길 28,190-2,글로벌 캠퍼스 푸르지오,월세,39.95,201801,3,1000,65,25,2013.0,송도동,2018,오피스텔
2,인천광역시 연수구 송도동,송도문화로28번길 28,190-2,글로벌 캠퍼스 푸르지오,전세,46.32,201801,8,18000,0,5,2013.0,송도동,2018,오피스텔
3,인천광역시 연수구 송도동,송도문화로28번길 28,190-2,글로벌 캠퍼스 푸르지오,전세,24.25,201801,10,13000,0,25,2013.0,송도동,2018,오피스텔
4,인천광역시 연수구 송도동,송도문화로28번길 28,190-2,글로벌 캠퍼스 푸르지오,전세,39.95,201801,13,17000,0,24,2013.0,송도동,2018,오피스텔


## 전월세 분리

In [79]:
officetel_rent_long = officetel_rent[officetel_rent['rent_type'] == '전세'].reset_index()
officetel_rent_month = officetel_rent[officetel_rent['rent_type'] == '월세'].reset_index()

## 전세

In [80]:
officetel_rent_long.head()

,index,bjd_nm,road_nm,bunji,cmplx_nm,rent_type,exclusive_area,cntrc_ym,cntrc_day,deposit,m_rent,floor,cnstr_year,dong,cntrc_year,type
0,0,인천광역시 연수구 송도동,송도문화로28번길 28,190-2,글로벌 캠퍼스 푸르지오,전세,49.32,201801,2,17500,0,5,2013.0,송도동,2018,오피스텔
1,2,인천광역시 연수구 송도동,송도문화로28번길 28,190-2,글로벌 캠퍼스 푸르지오,전세,46.32,201801,8,18000,0,5,2013.0,송도동,2018,오피스텔
2,3,인천광역시 연수구 송도동,송도문화로28번길 28,190-2,글로벌 캠퍼스 푸르지오,전세,24.25,201801,10,13000,0,25,2013.0,송도동,2018,오피스텔
3,4,인천광역시 연수구 송도동,송도문화로28번길 28,190-2,글로벌 캠퍼스 푸르지오,전세,39.95,201801,13,17000,0,24,2013.0,송도동,2018,오피스텔
4,5,인천광역시 연수구 송도동,송도문화로28번길 28,190-2,글로벌 캠퍼스 푸르지오,전세,49.63,201801,13,18000,0,9,2013.0,송도동,2018,오피스텔


In [81]:
deposit_per_area = [] # 제곱미터당 보증금
for i in range(len(officetel_rent_long)):
    deposit_per_area.append(officetel_rent_long['deposit'][i] / officetel_rent_long['exclusive_area'][i])

officetel_rent_long['deposit_per_area'] = deposit_per_area

In [82]:
officetel_rent_long.head()

,index,bjd_nm,road_nm,bunji,cmplx_nm,rent_type,exclusive_area,cntrc_ym,cntrc_day,deposit,m_rent,floor,cnstr_year,dong,cntrc_year,type,deposit_per_area
0,0,인천광역시 연수구 송도동,송도문화로28번길 28,190-2,글로벌 캠퍼스 푸르지오,전세,49.32,201801,2,17500,0,5,2013.0,송도동,2018,오피스텔,354.825629
1,2,인천광역시 연수구 송도동,송도문화로28번길 28,190-2,글로벌 캠퍼스 푸르지오,전세,46.32,201801,8,18000,0,5,2013.0,송도동,2018,오피스텔,388.601036
2,3,인천광역시 연수구 송도동,송도문화로28번길 28,190-2,글로벌 캠퍼스 푸르지오,전세,24.25,201801,10,13000,0,25,2013.0,송도동,2018,오피스텔,536.082474
3,4,인천광역시 연수구 송도동,송도문화로28번길 28,190-2,글로벌 캠퍼스 푸르지오,전세,39.95,201801,13,17000,0,24,2013.0,송도동,2018,오피스텔,425.531915
4,5,인천광역시 연수구 송도동,송도문화로28번길 28,190-2,글로벌 캠퍼스 푸르지오,전세,49.63,201801,13,18000,0,9,2013.0,송도동,2018,오피스텔,362.683861


### 최종 데이터프레임

In [83]:
officetel_rent_long_final = pd.DataFrame({'end_nm': officetel_rent_long['dong'], 'base_date': officetel_rent_long['cntrc_year'],
                                      'deposit_per_area': officetel_rent_long['deposit_per_area'], 'exclusive_area': officetel_rent_long['exclusive_area'],
                                       'cnstr_year': officetel_rent_long['cnstr_year'].astype(int),
                                 'floor': officetel_rent_long['floor'], 'type': officetel_rent_long['type']})

In [84]:
officetel_rent_long_final.head()

,end_nm,base_date,deposit_per_area,exclusive_area,cnstr_year,floor,type
0,송도동,2018,354.825629,49.32,2013,5,오피스텔
1,송도동,2018,388.601036,46.32,2013,5,오피스텔
2,송도동,2018,536.082474,24.25,2013,25,오피스텔
3,송도동,2018,425.531915,39.95,2013,24,오피스텔
4,송도동,2018,362.683861,49.63,2013,9,오피스텔


## 월세

In [85]:
officetel_rent_month.head()

,index,bjd_nm,road_nm,bunji,cmplx_nm,rent_type,exclusive_area,cntrc_ym,cntrc_day,deposit,m_rent,floor,cnstr_year,dong,cntrc_year,type
0,1,인천광역시 연수구 송도동,송도문화로28번길 28,190-2,글로벌 캠퍼스 푸르지오,월세,39.95,201801,3,1000,65,25,2013.0,송도동,2018,오피스텔
1,6,인천광역시 연수구 송도동,송도문화로28번길 28,190-2,글로벌 캠퍼스 푸르지오,월세,29.76,201801,17,500,50,20,2013.0,송도동,2018,오피스텔
2,10,인천광역시 연수구 송도동,송도문화로28번길 28,190-2,글로벌 캠퍼스 푸르지오,월세,25.16,201801,20,500,50,28,2013.0,송도동,2018,오피스텔
3,11,인천광역시 연수구 송도동,송도문화로28번길 28,190-2,글로벌 캠퍼스 푸르지오,월세,25.16,201801,24,500,50,15,2013.0,송도동,2018,오피스텔
4,13,인천광역시 연수구 송도동,송도문화로28번길 28,190-2,글로벌 캠퍼스 푸르지오,월세,24.25,201801,30,500,50,15,2013.0,송도동,2018,오피스텔


### 최종 데이터프레임

In [86]:
officetel_rent_month_final = pd.DataFrame({'end_nm': officetel_rent_month['dong'], 'base_date': officetel_rent_month['cntrc_year'], 
                                      'deposit': officetel_rent_month['deposit'], 'm_rent': officetel_rent_month['m_rent'],
                                       'exclusive_area': officetel_rent_month['exclusive_area'],
                                      'cnstr_year': officetel_rent_month['cnstr_year'].astype(int),
                                 'floor': officetel_rent_month['floor'], 'type': officetel_rent_month['type']})

In [87]:
officetel_rent_month_final.head()

,end_nm,base_date,deposit,m_rent,exclusive_area,cnstr_year,floor,type
0,송도동,2018,1000,65,39.95,2013,25,오피스텔
1,송도동,2018,500,50,29.76,2013,20,오피스텔
2,송도동,2018,500,50,25.16,2013,28,오피스텔
3,송도동,2018,500,50,25.16,2013,15,오피스텔
4,송도동,2018,500,50,24.25,2013,15,오피스텔


# 전세, 월세 최종 데이터프레임

## 전세

In [88]:
rent_long_final = pd.concat([apart_rent_long_final, villa_rent_long_final, officetel_rent_long_final], ignore_index=True)
rent_long_final

,end_nm,base_date,deposit_per_area,exclusive_area,cnstr_year,floor,type
0,동춘동,2018,247.218789,84.945,1994,3.0,아파트
1,동춘동,2018,235.446465,84.945,1994,4.0,아파트
2,동춘동,2018,235.446465,84.945,1994,2.0,아파트
3,동춘동,2018,189.158834,84.585,1994,2.0,아파트
4,동춘동,2018,265.763072,75.255,1994,6.0,아파트
...,...,...,...,...,...,...,...
38226,송도동,2021,508.543531,49.160,2011,6.0,오피스텔
38227,송도동,2021,363.100081,49.160,2011,9.0,오피스텔
38228,송도동,2021,508.543531,49.160,2011,8.0,오피스텔
38229,송도동,2021,508.543531,49.160,2011,5.0,오피스텔


In [89]:
rent_long_final.to_csv('최종_공동주택_전세_실거래가.csv')

## 월세

In [90]:
rent_month_final = pd.concat([apart_rent_month_final, villa_rent_month_final, officetel_rent_month_final], ignore_index=True)
rent_month_final

,end_nm,base_date,deposit,m_rent,exclusive_area,cnstr_year,floor,type
0,동춘동,2018,2000,70,84.945,1994,2.0,아파트
1,동춘동,2018,2000,40,53.010,1993,6.0,아파트
2,동춘동,2018,500,15,52.140,1994,7.0,아파트
3,동춘동,2018,1000,57,52.140,1994,9.0,아파트
4,동춘동,2018,2000,60,52.140,1994,10.0,아파트
...,...,...,...,...,...,...,...,...
12392,연수동,2021,1000,35,26.980,2003,7.0,오피스텔
12393,연수동,2021,1500,30,29.110,2003,10.0,오피스텔
12394,연수동,2021,500,40,26.980,2003,12.0,오피스텔
12395,연수동,2021,100,18,10.660,2002,7.0,오피스텔


In [91]:
rent_month_final.to_csv('최종_공동주택_월세_실거래가.csv')

# 전처리 - 25_연수구_분양입주권_실거래가

In [92]:
residency_trade.head()

,bjd_nm,bunji,cmplx_nm,exclusive_area,trade_price,floor,cntrc_ym,cntrc_day,right_type
0,인천광역시 연수구 송도동,18-6,더샵그린워크1차,59.9201,31133,20,201301,4,분
1,인천광역시 연수구 송도동,18-6,더샵그린워크1차,72.5113,35518,11,201301,7,분
2,인천광역시 연수구 송도동,18-6,더샵그린워크1차,59.9201,31311,9,201301,8,분
3,인천광역시 연수구 송도동,18-6,더샵그린워크1차,84.8682,41115,9,201301,21,분
4,인천광역시 연수구 송도동,18-6,더샵그린워크1차,101.9294,53258,14,201301,28,분


In [93]:
residency_trade.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21371 entries, 0 to 21370
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   bjd_nm          21371 non-null  object 
 1   bunji           21371 non-null  object 
 2   cmplx_nm        21371 non-null  object 
 3   exclusive_area  21371 non-null  float64
 4   trade_price     21371 non-null  int64  
 5   floor           21371 non-null  int64  
 6   cntrc_ym        21371 non-null  int64  
 7   cntrc_day       21371 non-null  int64  
 8   right_type      21371 non-null  object 
dtypes: float64(1), int64(4), object(4)
memory usage: 1.5+ MB


In [94]:
dong = []
for i in residency_trade['bjd_nm']:
    dong.append(i[-3:])
residency_trade['dong'] = dong

cntrc = []
for i in residency_trade['cntrc_ym']:
    cntrc.append(str(i)[0:4])

cntrc_year = []
for year in cntrc:
    cntrc_year.append(int(year))
residency_trade['cntrc_year'] = cntrc_year

price_per_area = [] # 제곱미터당 가격
for i in range(len(residency_trade)):
    price_per_area.append( residency_trade['trade_price'][i] / residency_trade['exclusive_area'][i] )
residency_trade['price_per_area'] = price_per_area

In [95]:
residency_trade.head()

,bjd_nm,bunji,cmplx_nm,exclusive_area,trade_price,floor,cntrc_ym,cntrc_day,right_type,dong,cntrc_year,price_per_area
0,인천광역시 연수구 송도동,18-6,더샵그린워크1차,59.9201,31133,20,201301,4,분,송도동,2013,519.575234
1,인천광역시 연수구 송도동,18-6,더샵그린워크1차,72.5113,35518,11,201301,7,분,송도동,2013,489.827103
2,인천광역시 연수구 송도동,18-6,더샵그린워크1차,59.9201,31311,9,201301,8,분,송도동,2013,522.545857
3,인천광역시 연수구 송도동,18-6,더샵그린워크1차,84.8682,41115,9,201301,21,분,송도동,2013,484.457076
4,인천광역시 연수구 송도동,18-6,더샵그린워크1차,101.9294,53258,14,201301,28,분,송도동,2013,522.498906


In [96]:
residency_trade = residency_trade[(residency_trade['cntrc_year'] == 2018)
                          | (residency_trade['cntrc_year'] == 2019)
                          | (residency_trade['cntrc_year']==2020)
                          | (residency_trade['cntrc_year']==2021)]
                          
residency_trade.reset_index(inplace=True)
residency_trade.drop('index', axis=1, inplace=True)
residency_trade.head()

,bjd_nm,bunji,cmplx_nm,exclusive_area,trade_price,floor,cntrc_ym,cntrc_day,right_type,dong,cntrc_year,price_per_area
0,인천광역시 연수구 동춘동,BL-10,송도 동일하이빌 파크레인,93.7025,40060,7,201801,11,분,동춘동,2018,427.523278
1,인천광역시 연수구 동춘동,BL-10,송도 동일하이빌 파크레인,66.9470,32423,16,201801,16,분,동춘동,2018,484.308483
2,인천광역시 연수구 동춘동,BL-10,송도 동일하이빌 파크레인,74.2443,35408,22,201801,22,분,동춘동,2018,476.912032
3,인천광역시 연수구 동춘동,BL-10,송도 동일하이빌 파크레인,84.9044,38085,15,201801,22,분,동춘동,2018,448.563325
4,인천광역시 연수구 동춘동,BL-10,송도 동일하이빌 파크레인,74.2443,34714,11,201801,23,분,동춘동,2018,467.564513


## 최종 데이터프레임

In [97]:
residency_trade_final = pd.DataFrame({'end_nm':residency_trade['dong'], 'base_date': residency_trade['cntrc_year'],
                                      'price_per_area': residency_trade['price_per_area'].round(3),
                                 'exclusive_area': residency_trade['exclusive_area'], 
                                  'floor': residency_trade['floor'],
                                  'right_type': residency_trade['right_type']})

In [98]:
residency_trade_final.head()

,end_nm,base_date,price_per_area,exclusive_area,floor,right_type
0,동춘동,2018,427.523,93.7025,7,분
1,동춘동,2018,484.308,66.9470,16,분
2,동춘동,2018,476.912,74.2443,22,분
3,동춘동,2018,448.563,84.9044,15,분
4,동춘동,2018,467.565,74.2443,11,분


In [99]:
residency_trade_final.to_csv('최종_분양입주권_실거래가.csv')

# 전처리 - 24_연수구_토지_실거래가

In [100]:
land_trade.head()

,bjd_nm,bunji,lot_purpose,use_area,road_cndtn,cntrc_ym,cntrc_day,cntrc_area,trade_price,share_type
0,인천광역시 연수구 동춘동,산1*,임야,보전녹지지역,-,201301,4,2083.00,35000,
1,인천광역시 연수구 옥련동,5**,대,일반상업지역,8m미만,201301,7,423.00,79400,
2,인천광역시 연수구 송도동,2*,대,준주거지역,25m미만,201301,10,165.48,46000,지분
3,인천광역시 연수구 송도동,*,대,준주거지역,25m미만,201301,15,1133.90,397999,
4,인천광역시 연수구 선학동,1**,답,개발제한구역,25m이상,201301,16,45.00,2421,


In [101]:
land_trade['use_area'].unique()

array(['보전녹지지역', '일반상업지역', '준주거지역', '개발제한구역', '제2종일반주거지역', '제3종일반주거지역',
       '중심상업지역', '제1종일반주거지역', '준공업지역', '자연녹지지역', '근린상업지역', '일반주거지역',
       '용도미지정'], dtype=object)

In [102]:
land_trade.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2459 entries, 0 to 2458
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   bjd_nm       2459 non-null   object 
 1   bunji        2459 non-null   object 
 2   lot_purpose  2459 non-null   object 
 3   use_area     2459 non-null   object 
 4   road_cndtn   2459 non-null   object 
 5   cntrc_ym     2459 non-null   int64  
 6   cntrc_day    2459 non-null   int64  
 7   cntrc_area   2459 non-null   float64
 8   trade_price  2459 non-null   int64  
 9   share_type   2459 non-null   object 
dtypes: float64(1), int64(3), object(6)
memory usage: 192.2+ KB


In [103]:
dong = []
for i in land_trade['bjd_nm']:
    dong.append(i[-3:])
land_trade['dong'] = dong

cntrc = []
for i in land_trade['cntrc_ym']:
    cntrc.append(str(i)[0:4])

cntrc_year = []
for year in cntrc:
    cntrc_year.append(int(year))
land_trade['cntrc_year'] = cntrc_year

price_per_area = [] # 제곱미터당 가격
for i in range(len(land_trade)):
    price_per_area.append( land_trade['trade_price'][i] / land_trade['cntrc_area'][i] )
land_trade['price_per_area'] = price_per_area

In [104]:
land_trade = land_trade[(land_trade['cntrc_year'] == 2018)
                          | (land_trade['cntrc_year'] == 2019)
                          | (land_trade['cntrc_year']==2020)
                          | (land_trade['cntrc_year']==2021)]
                          
land_trade.reset_index(inplace=True)
land_trade.drop('index', axis=1, inplace=True)
land_trade.head()

,bjd_nm,bunji,lot_purpose,use_area,road_cndtn,cntrc_ym,cntrc_day,cntrc_area,trade_price,share_type,dong,cntrc_year,price_per_area
0,인천광역시 연수구 옥련동,3**,전,제1종일반주거지역,12m미만,201801,5,71.00,11000,,옥련동,2018,154.929577
1,인천광역시 연수구 송도동,2*,대,준주거지역,25m미만,201801,8,168.76,65000,지분,송도동,2018,385.162361
2,인천광역시 연수구 선학동,5*,전,개발제한구역,8m미만,201801,10,894.00,55600,,선학동,2018,62.192394
3,인천광역시 연수구 동춘동,산6*,임야,제1종일반주거지역,-,201801,11,506.00,51040,,동춘동,2018,100.869565
4,인천광역시 연수구 동춘동,산6*,임야,제1종일반주거지역,-,201801,11,722.00,81000,,동춘동,2018,112.188366


## 최종 데이터프레임

In [105]:
land_trade_final = pd.DataFrame({'end_nm':land_trade['dong'], 'base_date': land_trade['cntrc_year'],
                                 'price_per_area': land_trade['price_per_area'].round(3), 'cntrc_area': land_trade['cntrc_area'], 
                                 'lot_purpose': land_trade['lot_purpose'], 'use_area': land_trade['use_area'], 'road_cndtn': land_trade['road_cndtn']
                                 })

In [106]:
land_trade_final.head()

,end_nm,base_date,price_per_area,cntrc_area,lot_purpose,use_area,road_cndtn
0,옥련동,2018,154.930,71.00,전,제1종일반주거지역,12m미만
1,송도동,2018,385.162,168.76,대,준주거지역,25m미만
2,선학동,2018,62.192,894.00,전,개발제한구역,8m미만
3,동춘동,2018,100.870,506.00,임야,제1종일반주거지역,-
4,동춘동,2018,112.188,722.00,임야,제1종일반주거지역,-


In [107]:
land_trade_final.to_csv('최종_토지_실거래가.csv')